In [1]:
from glob import glob

import numpy as np
from tifffile.tifffile import imread

import iob_ia.utils.filter_labels as fl
import iob_ia.utils.measure as meas
from iob_ia.utils import visualise

C:\Users\Microscopy\.conda\envs\iob-ia\lib\site-packages\cupy\_environment.py:217: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [2]:
full3D = "G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 2\\Testing\\"
every3rd3D = "G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 2\\Testing\\every3slice_4.38um\\"
every6th3D = "G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 2\\Testing\\every6slice_8.76um\\"
full2D = "G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 2\\Testing\\2D_z_step1.46um_2Dprediction40um\\"
every3rd2D = "G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 2\\Testing\\2D_z_step4.38um_2Dprediction40um\\"
every6th2D = "G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 2\\Testing\\2D_z_step8.76um_2Dprediction40um\\"

original_pixelsize = (1.46, 0.65, 0.65) # because images saved with python have no metadata
pix_3d_3rd = (4.38, 0.65, 0.65)
pix_3d_6th = (8.76, 0.65, 0.65)
pix_2d = (40, 0.65, 0.65)

In [3]:
def visualise_all(path: str, image_info: str, scale: tuple) -> tuple:
    """
    Shows the images in napari.

    returns: (gfp-channel, GFP-mask, rfp-channel, RFP-mask)
    """
    paths = glob(path + '*.tif')

    gfp = imread(find_path(paths, '-C2.'))
    g_mask = imread(find_path(paths, '-C2__cp-mask'))
    rfp = imread(find_path(paths, '-C3.'))
    r_mask = imread(find_path(paths, '-C3__cp-mask'))
    visualise.add_image(gfp, image_info + '-GFP', colormap='green', scale=scale)
    visualise.add_image(rfp, image_info + '-RFP', colormap='red', scale=scale)
    visualise.add_labels(g_mask, image_info + '-GFP-mask', scale=scale)
    visualise.add_labels(r_mask, image_info + '-RFP-mask', scale=scale)
    return gfp, g_mask, rfp, r_mask


def find_path(path: list[str], search_str: str) -> str:
    """
    Find paths containing a specific string.

    Finds the first path in the list of paths that contains the
    search_str.
    """
    for p in path:
        if search_str in p:
            #print(p)
            return p
    raise RuntimeError(f'could not find path contining <{search_str}>.')

def filter_doublePos_vis(images: list, filter_g: dict, filter_r: dict, name: str, voxel: tuple):
    """
    Filter labels by properties and show the filtered images (gfp and rfp objects).

    show double positive objects (e.g. gfp objects that have >50% overlap with rfp objects = rfp+gfp)

    returns: filtered-gfp, filtered-rfp, rfp+gfp, gfp+rfp
    """
    props_gfp = fl.measure_label_props(images[1], images[0])
    props_rfp = fl.measure_label_props(images[3], images[2])

    mask_gfp_filtered, props_gfp_filtered = fl.filter_labels_by_property(images[1], props_gfp, filter_g)
    mask_rfp_filtered, props_rfp_filtered = fl.filter_labels_by_property(images[3], props_rfp, filter_r)

    rfp_pos_gfp_objects = meas.overlapping_labels(mask_gfp_filtered, mask_rfp_filtered)
    gfp_pos_rfp_objects = meas.overlapping_labels(mask_rfp_filtered, mask_gfp_filtered)

    visualise.add_labels(mask_gfp_filtered, name + '-gfp_mask_filtered', scale=voxel)
    visualise.add_labels(mask_rfp_filtered, name + '-rfp_mask_filtered', scale=voxel)
    visualise.add_labels(rfp_pos_gfp_objects, name + '-rfp+gfp', scale=voxel)
    visualise.add_labels(gfp_pos_rfp_objects, name + '-gfp+rfp', scale=voxel)
    return mask_gfp_filtered, mask_rfp_filtered, rfp_pos_gfp_objects, gfp_pos_rfp_objects

In [4]:
# 3D
full3d_imgs = visualise_all(full3D, 'Full3D', original_pixelsize)

In [8]:
third3d_imgs = visualise_all(every3rd3D, '3rdSlice3D', pix_3d_3rd)

In [10]:
sixth3d_imgs = visualise_all(every6th3D, '6thSlice3D', pix_3d_6th)

In [11]:
# 2D
full2d_imgs = visualise_all(full2D, 'Full2D', pix_2d)

In [12]:
third2d_imgs = visualise_all(every3rd2D, '3rdSlice2D', pix_2d)

In [9]:
sixth2d_imgs = visualise_all(every6th2D, '6thSlice2D', pix_2d)

Example of filtering objects and checking for overlapping objects

In [5]:
# Filtering dictionaries
f_full3d_gfp = {
    'area': [700, 35000],
    'intensity_mean': [200, None]
}
f_full3d_rfp = {
    'area': [700, 35000],
    'intensity_mean': [150, None],
    'intensity_max': [None, 15000]
}


f_3rd_gfp = {
    'area': [200, None],
    'intensity_mean': [200, None]
}
f_3rd_rfp = {
    'area': [200, None],
    'intensity_mean': [150, None],
    'intensity_max': [None, 15000]
}

f_6th_gfp = {
    'area': [200, None],
    'intensity_mean': [200, None]
}
f_6th_rfp = {
    'area': [200, None],
    'intensity_mean': [150, None],
    'intensity_max': [None, 15000]
}

f_2d_gfp = {
    'area': [100, 550],
    'intensity_mean': [200, None]
}
f_2d_rfp = {
    'area': [100, 550],
    'intensity_mean': [150, None],
    'intensity_max': [None, 4000]
}

In [6]:
# example for full 3d
full3d_masks = filter_doublePos_vis(full3d_imgs, f_full3d_gfp, f_full3d_rfp, 'Full3D', original_pixelsize)

In [12]:
third3d_masks = filter_doublePos_vis(third3d_imgs, f_3rd_gfp, f_3rd_rfp, '3rdSlice3D', pix_3d_3rd)

In [13]:
sixth3d_masks = filter_doublePos_vis(sixth3d_imgs, f_6th_gfp, f_6th_rfp, '6thSlice3D', pix_3d_6th)

In [14]:
full2d_masks = filter_doublePos_vis(full2d_imgs, f_2d_gfp, f_2d_rfp, 'Full2D', pix_2d)

In [15]:
third2D_masks = filter_doublePos_vis(third2d_imgs, f_2d_gfp, f_2d_rfp, '3rdSlice2D', pix_2d)

In [16]:
sixt3D_masks = filter_doublePos_vis(sixth2d_imgs, f_2d_gfp, f_2d_rfp, '6thSlice2D', pix_2d)

In [1]:
from pathlib import Path
from time import time

from cellpose import core, models
from cellpose.io import logger_setup
from tifffile.tifffile import imwrite



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.11.11 
torch version:  	2.7.0+cu118! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




## Testing minimal resolution on Organoid 2, for 2D and 3D
Make sure that **cellpose >4 is running (e.g. 4.0.6)**

10x

pixel size xy = 0.65

pixel size z = 1.46

In [2]:
path = 'G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/*.tif'

xy_res = 0.65
z_res = 1.46

In [3]:
use_gpu = core.use_gpu()
logger_setup()
channels = [0,0]
model = models.CellposeModel(gpu=use_gpu, model_type='cpsam')

2025-08-05 10:50:58,200 [INFO] WRITING LOG OUTPUT TO C:\Users\Microscopy\.cellpose\run.log
2025-08-05 10:50:58,201 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.11.11 
torch version:  	2.7.0+cu118
2025-08-05 10:50:58,201 [WARNING] model_type argument is not used in v4.0.1+. Ignoring this argument...
2025-08-05 10:50:58,202 [INFO] ** TORCH CUDA version installed and working. **
2025-08-05 10:50:58,203 [INFO] >>>> using GPU (CUDA)
2025-08-05 10:50:59,638 [INFO] >>>> loading model C:\Users\Microscopy\.cellpose\models\cpsam


In [6]:
glob(path)

creating new log file
2025-08-05 10:52:30,038 [INFO] WRITING LOG OUTPUT TO C:\Users\Microscopy\.cellpose\run.log
2025-08-05 10:52:30,038 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.11.11 
torch version:  	2.7.0+cu118


(<Logger cellpose.io (INFO)>,
 WindowsPath('C:/Users/Microscopy/.cellpose/run.log'))

# 3D

In [7]:
# Predict full images
logger_setup()
start = time()
for p in glob(path):
    if 'mask' not in p:
        img = imread(p)
        print('Processing:', p)

        result = model.eval(
            img, channels=channels,
            diameter=15,
            do_3D=True,
            z_axis=0,
        )

        imwrite(p.replace('.tif', '__cp-mask.tif'), result[0])
        print('    >mask saved')
        del img
        del result
        print('>>>finished processing:', p, "\n>>>Current processing time:", time()-start)

print('Took:', time()-start)

# I accidentaly run the cell again (then aborted), from timestamps, the C3 image took 69min

creating new log file
2025-08-05 10:53:00,019 [INFO] WRITING LOG OUTPUT TO C:\Users\Microscopy\.cellpose\run.log
2025-08-05 10:53:00,019 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.11.11 
torch version:  	2.7.0+cu118
Processing: G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2.tif
2025-08-05 10:53:00,429 [WARNING] channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
2025-08-05 10:53:27,087 [INFO] running YX: 373 planes of size (2676, 2108)
2025-08-05 11:13:25,190 [INFO] 100%|##########| 373/373 [19:58<00:00,  3.21s/it]
2025-08-05 11:13:34,923 [INFO] running ZY: 2676 planes of size (373, 2108)
2025-08-05 11:36:07,878 [INFO] 100%|##########| 2676/2676 [22:32<00:00,  1.98it/s]
2025-08-05 11:36:17,506 [INFO] running ZX: 2108 planes of size (373, 2676)
2025-08-05 11:59:44,118 [INFO] 100%|##########| 2108/2108 [23:26<00:00,  1.50it/s]
2025-08-05 12:00:20,450 [INFO] netwo

In [8]:
# Predict every 3rd slice
logger_setup()
glob_paths = glob(path)
print('images in folder:')
for p in glob_paths:
    print('-', p)

start = time()
for p in glob(path):
    if 'mask' not in p:
        img = imread(p)
        #print('original image shape', img.shape)
        print('Processing:', p)

        # Take only every n-th slice
        every_n_slice = 3
        img = img[::every_n_slice,:,:]
        #print('every 3rd slice shape:', img.shape)

        # Create output fodler
        out_dir = Path(p).parent
        z_step = z_res * every_n_slice
        out_dir = Path.joinpath(out_dir, f'every3slice_{z_step}um')
        if not Path.exists(out_dir):
            Path.mkdir(out_dir)
            print('Created output folder:', out_dir)

        # Save raw image too
        file_name = Path(p).name
        file_name.replace('.tif', f'_{z_step}um.tif')
        imwrite(Path.joinpath(out_dir, file_name), img)
        print('Saved raw image to:', Path.joinpath(out_dir, file_name))

        result = model.eval(
            img, channels=channels,
            diameter=15,
            do_3D=True,
            z_axis=0,
        )

        file_name = file_name.replace('.tif', '__cp-mask.tif')
        imwrite(Path.joinpath(out_dir, file_name), result[0])
        print('    >mask saved to:', Path.joinpath(out_dir, file_name))
        del img
        del result
        print('>>>finished processing:', p, "\n>>>Current processing time:", time()-start)

print('Took:', time()-start)

creating new log file
2025-08-05 13:23:33,520 [INFO] WRITING LOG OUTPUT TO C:\Users\Microscopy\.cellpose\run.log
2025-08-05 13:23:33,520 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.11.11 
torch version:  	2.7.0+cu118
images in folder:
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2__cp-mask.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C3.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C3__cp-mask.tif
Processing: G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2.tif
Saved raw image to: G:\20241120_IOB_Magdalena\20250520_TestClearing\Organoid 2\Testing\every3slice_4.38um\20250520_10x-C2.tif
2025-08-05 13:23:34,792 [WARNING] channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels wil

In [9]:
logger_setup()
# Predict every 6th slice
glob_paths = glob(path)
print('images in folder:')
for p in glob_paths:
    print('-', p)

# Take only every n-th slice
every_n_slice = 6

start = time()
for p in glob(path):
    if 'mask' not in p:
        img = imread(p)
        #print('original image shape', img.shape)
        print('Processing:', p)

        img = img[::every_n_slice,:,:]
        #print('every 3rd slice shape:', img.shape)

        # Create output fodler
        out_dir = Path(p).parent
        z_step = z_res * every_n_slice
        out_dir = Path.joinpath(out_dir, f'every6slice_{z_step}um')
        if not Path.exists(out_dir):
            Path.mkdir(out_dir)
            print('Created output folder:', out_dir)

        # Save raw image too
        file_name = Path(p).name
        file_name.replace('.tif', f'_{z_step}um.tif')
        imwrite(Path.joinpath(out_dir, file_name), img)
        print('Saved raw image to:', Path.joinpath(out_dir, file_name))

        result = model.eval(
            img, channels=channels,
            diameter=15,
            do_3D=True,
            z_axis=0,
        )

        file_name = file_name.replace('.tif', '__cp-mask.tif')
        imwrite(Path.joinpath(out_dir, file_name), result[0])
        print('    >mask saved to:', Path.joinpath(out_dir, file_name))
        del img
        del result
        print('>>>finished processing:', p, "\n>>>Current processing time:", time()-start)

print('Took:', time()-start)

creating new log file
2025-08-05 14:25:18,095 [INFO] WRITING LOG OUTPUT TO C:\Users\Microscopy\.cellpose\run.log
2025-08-05 14:25:18,095 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.11.11 
torch version:  	2.7.0+cu118
images in folder:
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2__cp-mask.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C3.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C3__cp-mask.tif
Processing: G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2.tif
Saved raw image to: G:\20241120_IOB_Magdalena\20250520_TestClearing\Organoid 2\Testing\every6slice_8.76um\20250520_10x-C2.tif
2025-08-05 14:25:18,926 [WARNING] channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels wil

# 2D

make projections of substacks

In [51]:
logger_setup()
# Predict sub-stacks 40um, on "nyquist acquisition"
glob_paths = glob(path)
print('images in folder:')
for p in glob_paths:
    print('-', p)

# Take only every n-th slice
every_n_slice = 1

z_step = z_res * every_n_slice
# Take only every n-th slice
n_slices = int(round(40 / z_step, 0))

start = time()
for p in glob(path):
    if 'mask' not in p:
        img = imread(p)
        #print('original image shape', img.shape)
        print('Processing:', p)

        img = img[::every_n_slice,:,:]
        #print('every 3rd slice shape:', img.shape)

        # Create output fodler
        out_dir = Path(p).parent
        out_dir = Path.joinpath(out_dir, f'2D_z_step{z_step}um_2Dprediction40um')
        if not Path.exists(out_dir):
            Path.mkdir(out_dir)
            print('Created output folder:', out_dir)

        # Create substacks of the full stack and project them
        img_substacks = []
        mask_substaks= []
        n_label_max = 0
        for z in range(0, img.shape[0], int(n_slices)):
            # make substacks of n-slices sequentially, (if z>Z-dim, takes Z-dim)
            substack = np.max(img[z:z+n_slices, :, :], axis=0)
            img_substacks.append(substack)

            # predict current substack
            result = model.eval(
                substack, channels=channels,
                diameter=15,
                do_3D=False,
            )
            mask = np.where(result[0] != 0, result[0] + n_label_max, result[0])
            n_label_max = mask.max()
            mask_substaks.append(mask)

        # convert the sub-stack projections to single stack
        img_substacks = np.asarray(img_substacks)
        mask_substaks = np.asarray(mask_substaks)

        # save the images
        file_name = Path(p).name
        imwrite(Path.joinpath(out_dir, file_name), img_substacks)
        print('>>>Saved raw image to:', Path.joinpath(out_dir, file_name))
        file_name = file_name.replace('.tif', '__cp-mask.tif')
        imwrite(Path.joinpath(out_dir, file_name), mask_substaks)
        print('>>>Saved mask image to:', Path.joinpath(out_dir, file_name))
        print('>>>finished processing:', p, "\n>>>Current processing time:", time()-start)

print('Took:', time()-start)

images in folder:
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2__cp-mask.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C3.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C3__cp-mask.tif
Processing: G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2.tif
Created output folder: G:\20241120_IOB_Magdalena\20250520_TestClearing\Organoid 2\Testing\2D_z_step1.46um_2Dprediction40um
2025-07-30 14:51:57,713 [WARNING] channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
2025-07-30 14:52:01,463 [WARNING] Resizing is depricated in v4.0.1+
2025-07-30 14:52:01,570 [WARNING] channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
2025-07-30 14:52:05,148 [WARNING

In [52]:
logger_setup()
# Predict sub-stacks 40um, on "nyquist acquisition"
glob_paths = glob(path)
print('images in folder:')
for p in glob_paths:
    print('-', p)

# Take only every n-th slice
every_n_slice = 3

z_step = z_res * every_n_slice
# Take only every n-th slice
n_slices = int(round(40 / z_step, 0))

start = time()
for p in glob(path):
    if 'mask' not in p:
        img = imread(p)
        #print('original image shape', img.shape)
        print('Processing:', p, 'with image shape=', img.shape)

        img = img[::every_n_slice,:,:]
        print('took every', every_n_slice, 'th slice, image has shape =', img.shape)

        # Create output fodler
        out_dir = Path(p).parent
        out_dir = Path.joinpath(out_dir, f'2D_z_step{z_step}um_2Dprediction40um')
        if not Path.exists(out_dir):
            Path.mkdir(out_dir)
            print('Created output folder:', out_dir)

        # Create substacks of the full stack and project them
        img_substacks = []
        mask_substaks= []
        n_label_max = 0
        for z in range(0, img.shape[0], int(n_slices)):
            # make substacks of n-slices sequentially, (if z>Z-dim, takes Z-dim)
            print('Creating substack of slices:', z, '-', z+n_slices)
            substack = np.max(img[z:z+n_slices, :, :], axis=0)
            img_substacks.append(substack)

            # predict current substack
            result = model.eval(
                substack, channels=channels,
                diameter=15,
                do_3D=False,
            )
            mask = np.where(result[0] != 0, result[0] + n_label_max, result[0])
            n_label_max = mask.max()
            mask_substaks.append(mask)

        # convert the sub-stack projections to single stack
        img_substacks = np.asarray(img_substacks)
        mask_substaks = np.asarray(mask_substaks)

        # save the images
        file_name = Path(p).name
        imwrite(Path.joinpath(out_dir, file_name), img_substacks)
        print('>>>Saved raw image to:', Path.joinpath(out_dir, file_name))
        file_name = file_name.replace('.tif', '__cp-mask.tif')
        imwrite(Path.joinpath(out_dir, file_name), mask_substaks)
        print('>>>Saved mask image to:', Path.joinpath(out_dir, file_name))
        print('>>>finished processing:', p, "\n>>>Current processing time:", time()-start)

print('Took:', time()-start)

images in folder:
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2__cp-mask.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C3.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C3__cp-mask.tif
Processing: G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2.tif with image shape= (373, 1338, 1054)
took every 3 th slice, image has shape = (125, 1338, 1054)
Created output folder: G:\20241120_IOB_Magdalena\20250520_TestClearing\Organoid 2\Testing\2D_z_step4.38um_2Dprediction40um
Creating substack of slices: 0 - 9
2025-07-30 14:53:43,935 [WARNING] channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
2025-07-30 14:53:47,416 [WARNING] Resizing is depricated in v4.0.1+
Creating substack of slices: 9 - 18
2025-07-3

In [53]:
logger_setup()
# Predict sub-stacks 40um, on "nyquist acquisition"
glob_paths = glob(path)
print('images in folder:')
for p in glob_paths:
    print('-', p)

# Take only every n-th slice
every_n_slice = 6

z_step = z_res * every_n_slice
# Take only every n-th slice
n_slices = int(round(40 / z_step, 0))

start = time()
for p in glob(path):
    if 'mask' not in p:
        img = imread(p)
        #print('original image shape', img.shape)
        print('Processing:', p, 'with image shape=', img.shape)

        img = img[::every_n_slice,:,:]
        print('took every', every_n_slice, 'th slice, image has shape =', img.shape)

        # Create output fodler
        out_dir = Path(p).parent
        out_dir = Path.joinpath(out_dir, f'2D_z_step{z_step}um_2Dprediction40um')
        if not Path.exists(out_dir):
            Path.mkdir(out_dir)
            print('Created output folder:', out_dir)

        # Create substacks of the full stack and project them
        img_substacks = []
        mask_substaks= []
        n_label_max = 0
        for z in range(0, img.shape[0], int(n_slices)):
            # make substacks of n-slices sequentially, (if z>Z-dim, takes Z-dim)
            print('Creating substack of slices:', z, '-', z+n_slices)
            substack = np.max(img[z:z+n_slices, :, :], axis=0)
            img_substacks.append(substack)

            # predict current substack
            result = model.eval(
                substack, channels=channels,
                diameter=15,
                do_3D=False,
            )
            mask = np.where(result[0] != 0, result[0] + n_label_max, result[0])
            n_label_max = mask.max()
            mask_substaks.append(mask)

        # convert the sub-stack projections to single stack
        img_substacks = np.asarray(img_substacks)
        mask_substaks = np.asarray(mask_substaks)

        # save the images
        file_name = Path(p).name
        imwrite(Path.joinpath(out_dir, file_name), img_substacks)
        print('>>>Saved raw image to:', Path.joinpath(out_dir, file_name))
        file_name = file_name.replace('.tif', '__cp-mask.tif')
        imwrite(Path.joinpath(out_dir, file_name), mask_substaks)
        print('>>>Saved mask image to:', Path.joinpath(out_dir, file_name))
        print('>>>finished processing:', p, "\n>>>Current processing time:", time()-start)

print('Took:', time()-start)

images in folder:
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2__cp-mask.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C3.tif
- G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C3__cp-mask.tif
Processing: G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2.tif with image shape= (373, 1338, 1054)
took every 6 th slice, image has shape = (63, 1338, 1054)
Created output folder: G:\20241120_IOB_Magdalena\20250520_TestClearing\Organoid 2\Testing\2D_z_step8.76um_2Dprediction40um
Creating substack of slices: 0 - 5
2025-07-30 14:55:30,190 [WARNING] channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
2025-07-30 14:55:33,680 [WARNING] Resizing is depricated in v4.0.1+
Creating substack of slices: 5 - 10
2025-07-30

# Counting cells
--> should use the iob-ia functions to analyse this (that env does not yet have the newest cell pose)
for 2D min cell size should be around 80-600 pixels

for 3D filter with:
- full stack: area = 700-35000, C2-intensity_mean = 200-inf, C3-intensity_mean = 150-inf, C3-intensity_max = -inf - 15000
- 4.38 steps: area = 200-inf, C2-intensity_mean = 200-inf, C3-intensity_mean = 150-inf, C3-intensity_max = -inf - 15000
- 8.76 steps: area = 70-inf,  C2-intensity_mean = 200-inf, C3-intensity_mean = 150-inf, C3-intensity_max = -inf - 15000

In [7]:
%load_ext autoreload
%autoreload 2

In [11]:
from glob import glob
from pathlib import Path
from time import time

import numpy as np
from tifffile.tifffile import imread, imwrite



# 3D cell counts

In [12]:
print('Checking object counts of full stack segmenation...')
path_base = 'G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/'

xy_res = 0.65 # unused
z_res = 1.46  # unused

start = time()
# full stacks
img_gfp_path = ''
img_rfp_path = ''
mask_gfp_path = ''
mask_rfp_path = ''

for p in glob(path_base + '*.tif'):
    if 'C2.tif' in p:
        img_gfp_path = p
    if 'C3.tif' in p:
        img_rfp_path = p
    if 'C2__cp' in p:
        mask_gfp_path = p
    if 'C3__cp' in p:
        mask_rfp_path = p
print('Image paths:')
print(img_gfp_path)
print(img_rfp_path)
print(mask_gfp_path)
print(mask_rfp_path)

# read images
img_gfp = imread(img_gfp_path)
img_rfp = imread(img_rfp_path)
mask_gfp = imread(mask_gfp_path)
mask_rfp = imread(mask_rfp_path)

# Filter the labels
props_gfp = fl.measure_label_props(mask_gfp, img_gfp)
props_rfp = fl.measure_label_props(mask_rfp, img_rfp)

filter_gfp = {
    'area': [700, 35000],
    'intensity_mean': [200, None]
}
filter_rfp = {
    'area': [700, 35000],
    'intensity_mean': [150, None],
    'intensity_max': [None, 15000]
}
mask_gfp_filtered, props_gfp_filtered = fl.filter_labels_by_property(mask_gfp, props_gfp, filter_gfp)
mask_rfp_filtered, props_rfp_filtered = fl.filter_labels_by_property(mask_rfp, props_rfp, filter_rfp)

print('GFP mask had', len(np.unique(mask_gfp)) - 1, 'labels. After filtering:', len(np.unique(mask_gfp_filtered)) -1)
print('RFG mask had', len(np.unique(mask_rfp)) - 1, 'labels. After filtering:', len(np.unique(mask_rfp_filtered)) -1)

# TODO 'Binary feature extractor' on both channels seperately
rfp_pos_gfp_objects = meas.overlapping_labels(mask_gfp_filtered, mask_rfp_filtered)
gfp_pos_rfp_objects = meas.overlapping_labels(mask_rfp_filtered, mask_gfp_filtered)
print('Number of GFP objects also present as RFP objects:', len(np.unique(rfp_pos_gfp_objects)) - 1)
print('Number of RFP objects also present as GFP objects:', len(np.unique(gfp_pos_rfp_objects)) - 1)
print('Took:', time() - start)

Image paths:
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C3.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C2__cp-mask.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing\20250520_10x-C3__cp-mask.tif
GFP mask had 7478 labels. After filtering: 3146
RFG mask had 13368 labels. After filtering: 3529
Number of GFP objects also present as RFP objects: 828
Number of RFP objects also present as GFP objects: 1271
Took: 100.01316547393799


In [15]:
print('Checking object counts of every3slice stack (4.38um steps) segmenation...')
path_base = 'G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/every3slice_4.38um/'

xy_res = 0.65 # unused
z_res = 1.46  # unused

start = time()
# full stacks
img_gfp_path = ''
img_rfp_path = ''
mask_gfp_path = ''
mask_rfp_path = ''

for p in glob(path_base + '*.tif'):
    if 'C2.tif' in p:
        img_gfp_path = p
    if 'C3.tif' in p:
        img_rfp_path = p
    if 'C2__cp' in p:
        mask_gfp_path = p
    if 'C3__cp' in p:
        mask_rfp_path = p
print('Image paths:')
print(img_gfp_path)
print(img_rfp_path)
print(mask_gfp_path)
print(mask_rfp_path)

# read images
img_gfp = imread(img_gfp_path)
img_rfp = imread(img_rfp_path)
mask_gfp = imread(mask_gfp_path)
mask_rfp = imread(mask_rfp_path)

# Filter the labels
props_gfp = fl.measure_label_props(mask_gfp, img_gfp)
props_rfp = fl.measure_label_props(mask_rfp, img_rfp)

filter_gfp = {
    'area': [200, None],
    'intensity_mean': [200, None]
}
filter_rfp = {
    'area': [200, None],
    'intensity_mean': [150, None],
    'intensity_max': [None, 15000]
}
mask_gfp_filtered, props_gfp_filtered = fl.filter_labels_by_property(mask_gfp, props_gfp, filter_gfp)
mask_rfp_filtered, props_rfp_filtered = fl.filter_labels_by_property(mask_rfp, props_rfp, filter_rfp)

print('GFP mask had', len(np.unique(mask_gfp)) - 1, 'labels. After filtering:', len(np.unique(mask_gfp_filtered)) -1)
print('RFG mask had', len(np.unique(mask_rfp)) - 1, 'labels. After filtering:', len(np.unique(mask_rfp_filtered)) -1)

# TODO 'Binary feature extractor' on both channels seperately
rfp_pos_gfp_objects = meas.overlapping_labels(mask_gfp_filtered, mask_rfp_filtered)
gfp_pos_rfp_objects = meas.overlapping_labels(mask_rfp_filtered, mask_gfp_filtered)
print('Number of GFP objects also present as RFP objects:', len(np.unique(rfp_pos_gfp_objects)) - 1)
print('Number of RFP objects also present as GFP objects:', len(np.unique(gfp_pos_rfp_objects)) - 1)
print('Took:', time() - start)

Checking object counts of every3slice stack (4.38um steps) segmenation...
Image paths:
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/every3slice_4.38um\20250520_10x-C2.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/every3slice_4.38um\20250520_10x-C3.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/every3slice_4.38um\20250520_10x-C2__cp-mask.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/every3slice_4.38um\20250520_10x-C3__cp-mask.tif
GFP mask had 3520 labels. After filtering: 2891
RFG mask had 4053 labels. After filtering: 3305
Number of GFP objects also present as RFP objects: 797
Number of RFP objects also present as GFP objects: 1246
Took: 31.075172424316406


In [16]:
print('Checking object counts of every6slice stack (8.76um steps) segmenation...')
path_base = 'G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/every6slice_8.76um/'

xy_res = 0.65 # unused
z_res = 1.46  # unused

start = time()
# full stacks
img_gfp_path = ''
img_rfp_path = ''
mask_gfp_path = ''
mask_rfp_path = ''

for p in glob(path_base + '*.tif'):
    if 'C2.tif' in p:
        img_gfp_path = p
    if 'C3.tif' in p:
        img_rfp_path = p
    if 'C2__cp' in p:
        mask_gfp_path = p
    if 'C3__cp' in p:
        mask_rfp_path = p
print('Image paths:')
print(img_gfp_path)
print(img_rfp_path)
print(mask_gfp_path)
print(mask_rfp_path)

# read images
img_gfp = imread(img_gfp_path)
img_rfp = imread(img_rfp_path)
mask_gfp = imread(mask_gfp_path)
mask_rfp = imread(mask_rfp_path)

# Filter the labels
props_gfp = fl.measure_label_props(mask_gfp, img_gfp)
props_rfp = fl.measure_label_props(mask_rfp, img_rfp)

filter_gfp = {
    'area': [200, None],
    'intensity_mean': [200, None]
}
filter_rfp = {
    'area': [200, None],
    'intensity_mean': [150, None],
    'intensity_max': [None, 15000]
}
mask_gfp_filtered, props_gfp_filtered = fl.filter_labels_by_property(mask_gfp, props_gfp, filter_gfp)
mask_rfp_filtered, props_rfp_filtered = fl.filter_labels_by_property(mask_rfp, props_rfp, filter_rfp)

print('GFP mask had', len(np.unique(mask_gfp)) - 1, 'labels. After filtering:', len(np.unique(mask_gfp_filtered)) -1)
print('RFG mask had', len(np.unique(mask_rfp)) - 1, 'labels. After filtering:', len(np.unique(mask_rfp_filtered)) -1)

# TODO 'Binary feature extractor' on both channels seperately
rfp_pos_gfp_objects = meas.overlapping_labels(mask_gfp_filtered, mask_rfp_filtered)
gfp_pos_rfp_objects = meas.overlapping_labels(mask_rfp_filtered, mask_gfp_filtered)
print('Number of GFP objects also present as RFP objects:', len(np.unique(rfp_pos_gfp_objects)) - 1)
print('Number of RFP objects also present as GFP objects:', len(np.unique(gfp_pos_rfp_objects)) - 1)
print('Took:', time() - start)

Checking object counts of every6slice stack (8.76um steps) segmenation...
Image paths:
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/every6slice_8.76um\20250520_10x-C2.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/every6slice_8.76um\20250520_10x-C3.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/every6slice_8.76um\20250520_10x-C2__cp-mask.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/every6slice_8.76um\20250520_10x-C3__cp-mask.tif
GFP mask had 3042 labels. After filtering: 2204
RFG mask had 3309 labels. After filtering: 2249
Number of GFP objects also present as RFP objects: 496
Number of RFP objects also present as GFP objects: 817
Took: 18.681718349456787


# 2D cell counts

for 2D filter with
- full stack: area = 100-550, C2-intensity_mean = 200-inf, C3-intensity_mean = 150-inf, C3-intensity_max = -inf - 4000
- 4.38 steps: area = 100-550, C2-intensity_mean = 200-inf, C3-intensity_mean = 150-inf, C3-intensity_max = -inf - 4000
- 8.76 steps: area = 100-550, C2-intensity_mean = 200-inf, C3-intensity_mean = 150-inf, C3-intensity_max = -inf - 4000


In [17]:
print('Checking object counts of full substack (2D) segmentation...')
path_base = 'G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step1.46um_2Dprediction40um/'

xy_res = 0.65 # unused
z_res = 1.46  # unused

start = time()
# full stacks
img_gfp_path = ''
img_rfp_path = ''
mask_gfp_path = ''
mask_rfp_path = ''

for p in glob(path_base + '*.tif'):
    if 'C2.tif' in p:
        img_gfp_path = p
    if 'C3.tif' in p:
        img_rfp_path = p
    if 'C2__cp' in p:
        mask_gfp_path = p
    if 'C3__cp' in p:
        mask_rfp_path = p
print('Image paths:')
print(img_gfp_path)
print(img_rfp_path)
print(mask_gfp_path)
print(mask_rfp_path)

# read images
img_gfp = imread(img_gfp_path)
img_rfp = imread(img_rfp_path)
mask_gfp = imread(mask_gfp_path)
mask_rfp = imread(mask_rfp_path)

# Filter the labels
props_gfp = fl.measure_label_props(mask_gfp, img_gfp)
props_rfp = fl.measure_label_props(mask_rfp, img_rfp)

filter_gfp = {
    'area': [100, 550],
    'intensity_mean': [200, None]
}
filter_rfp = {
    'area': [100, 550],
    'intensity_mean': [150, None],
    'intensity_max': [None, 4000]
}
mask_gfp_filtered, props_gfp_filtered = fl.filter_labels_by_property(mask_gfp, props_gfp, filter_gfp)
mask_rfp_filtered, props_rfp_filtered = fl.filter_labels_by_property(mask_rfp, props_rfp, filter_rfp)

print('GFP mask had', len(np.unique(mask_gfp)) - 1, 'labels. After filtering:', len(np.unique(mask_gfp_filtered)) -1)
print('RFG mask had', len(np.unique(mask_rfp)) - 1, 'labels. After filtering:', len(np.unique(mask_rfp_filtered)) -1)

# TODO 'Binary feature extractor' on both channels seperately
rfp_pos_gfp_objects = meas.overlapping_labels(mask_gfp_filtered, mask_rfp_filtered)
gfp_pos_rfp_objects = meas.overlapping_labels(mask_rfp_filtered, mask_gfp_filtered)
print('Number of GFP objects also present as RFP objects:', len(np.unique(rfp_pos_gfp_objects)) - 1)
print('Number of RFP objects also present as GFP objects:', len(np.unique(gfp_pos_rfp_objects)) - 1)
print('Took:', time() - start)

Checking object counts of full substack (2D) segmentation...
Image paths:
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step1.46um_2Dprediction40um\20250520_10x-C2.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step1.46um_2Dprediction40um\20250520_10x-C3.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step1.46um_2Dprediction40um\20250520_10x-C2__cp-mask.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step1.46um_2Dprediction40um\20250520_10x-C3__cp-mask.tif
GFP mask had 5891 labels. After filtering: 5614
RFG mask had 7075 labels. After filtering: 6037
Number of GFP objects also present as RFP objects: 2357
Number of RFP objects also present as GFP objects: 2597
Took: 19.777377128601074


In [18]:
print('Checking object counts of full substack with 4.38um slices(2D) segmentation...')
path_base = 'G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step4.38um_2Dprediction40um/'

xy_res = 0.65 # unused
z_res = 1.46  # unused

start = time()
# full stacks
img_gfp_path = ''
img_rfp_path = ''
mask_gfp_path = ''
mask_rfp_path = ''

for p in glob(path_base + '*.tif'):
    if 'C2.tif' in p:
        img_gfp_path = p
    if 'C3.tif' in p:
        img_rfp_path = p
    if 'C2__cp' in p:
        mask_gfp_path = p
    if 'C3__cp' in p:
        mask_rfp_path = p
print('Image paths:')
print(img_gfp_path)
print(img_rfp_path)
print(mask_gfp_path)
print(mask_rfp_path)

# read images
img_gfp = imread(img_gfp_path)
img_rfp = imread(img_rfp_path)
mask_gfp = imread(mask_gfp_path)
mask_rfp = imread(mask_rfp_path)

# Filter the labels
props_gfp = fl.measure_label_props(mask_gfp, img_gfp)
props_rfp = fl.measure_label_props(mask_rfp, img_rfp)

filter_gfp = {
    'area': [100, 550],
    'intensity_mean': [200, None]
}
filter_rfp = {
    'area': [100, 550],
    'intensity_mean': [150, None],
    'intensity_max': [None, 4000]
}
mask_gfp_filtered, props_gfp_filtered = fl.filter_labels_by_property(mask_gfp, props_gfp, filter_gfp)
mask_rfp_filtered, props_rfp_filtered = fl.filter_labels_by_property(mask_rfp, props_rfp, filter_rfp)

print('GFP mask had', len(np.unique(mask_gfp)) - 1, 'labels. After filtering:', len(np.unique(mask_gfp_filtered)) -1)
print('RFG mask had', len(np.unique(mask_rfp)) - 1, 'labels. After filtering:', len(np.unique(mask_rfp_filtered)) -1)

# TODO 'Binary feature extractor' on both channels seperately
rfp_pos_gfp_objects = meas.overlapping_labels(mask_gfp_filtered, mask_rfp_filtered)
gfp_pos_rfp_objects = meas.overlapping_labels(mask_rfp_filtered, mask_gfp_filtered)
print('Number of GFP objects also present as RFP objects:', len(np.unique(rfp_pos_gfp_objects)) - 1)
print('Number of RFP objects also present as GFP objects:', len(np.unique(gfp_pos_rfp_objects)) - 1)
print('Took:', time() - start)

Checking object counts of full substack with 4.38um slices(2D) segmentation...
Image paths:
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step4.38um_2Dprediction40um\20250520_10x-C2.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step4.38um_2Dprediction40um\20250520_10x-C3.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step4.38um_2Dprediction40um\20250520_10x-C2__cp-mask.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step4.38um_2Dprediction40um\20250520_10x-C3__cp-mask.tif
GFP mask had 5611 labels. After filtering: 5324
RFG mask had 6614 labels. After filtering: 5535
Number of GFP objects also present as RFP objects: 2181
Number of RFP objects also present as GFP objects: 2405
Took: 18.680530548095703


In [19]:
print('Checking object counts of full substack with 8.76um slices (2D) segmentation...')
path_base = 'G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step8.76um_2Dprediction40um/'

xy_res = 0.65 # unused
z_res = 1.46  # unused

start = time()
# full stacks
img_gfp_path = ''
img_rfp_path = ''
mask_gfp_path = ''
mask_rfp_path = ''

for p in glob(path_base + '*.tif'):
    if 'C2.tif' in p:
        img_gfp_path = p
    if 'C3.tif' in p:
        img_rfp_path = p
    if 'C2__cp' in p:
        mask_gfp_path = p
    if 'C3__cp' in p:
        mask_rfp_path = p
print('Image paths:')
print(img_gfp_path)
print(img_rfp_path)
print(mask_gfp_path)
print(mask_rfp_path)

# read images
img_gfp = imread(img_gfp_path)
img_rfp = imread(img_rfp_path)
mask_gfp = imread(mask_gfp_path)
mask_rfp = imread(mask_rfp_path)

# Filter the labels
props_gfp = fl.measure_label_props(mask_gfp, img_gfp)
props_rfp = fl.measure_label_props(mask_rfp, img_rfp)

filter_gfp = {
    'area': [100, 550],
    'intensity_mean': [200, None]
}
filter_rfp = {
    'area': [100, 550],
    'intensity_mean': [150, None],
    'intensity_max': [None, 4000]
}
mask_gfp_filtered, props_gfp_filtered = fl.filter_labels_by_property(mask_gfp, props_gfp, filter_gfp)
mask_rfp_filtered, props_rfp_filtered = fl.filter_labels_by_property(mask_rfp, props_rfp, filter_rfp)

print('GFP mask had', len(np.unique(mask_gfp)) - 1, 'labels. After filtering:', len(np.unique(mask_gfp_filtered)) -1)
print('RFG mask had', len(np.unique(mask_rfp)) - 1, 'labels. After filtering:', len(np.unique(mask_rfp_filtered)) -1)

# TODO 'Binary feature extractor' on both channels seperately
rfp_pos_gfp_objects = meas.overlapping_labels(mask_gfp_filtered, mask_rfp_filtered)
gfp_pos_rfp_objects = meas.overlapping_labels(mask_rfp_filtered, mask_gfp_filtered)
print('Number of GFP objects also present as RFP objects:', len(np.unique(rfp_pos_gfp_objects)) - 1)
print('Number of RFP objects also present as GFP objects:', len(np.unique(gfp_pos_rfp_objects)) - 1)
print('Took:', time() - start)

Checking object counts of full substack with 8.76um slices (2D) segmentation...
Image paths:
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step8.76um_2Dprediction40um\20250520_10x-C2.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step8.76um_2Dprediction40um\20250520_10x-C3.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step8.76um_2Dprediction40um\20250520_10x-C2__cp-mask.tif
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/Testing/2D_z_step8.76um_2Dprediction40um\20250520_10x-C3__cp-mask.tif
GFP mask had 5021 labels. After filtering: 4734
RFG mask had 5814 labels. After filtering: 4823
Number of GFP objects also present as RFP objects: 1890
Number of RFP objects also present as GFP objects: 2071
Took: 16.621212482452393
